KNMI. (1911–2021b, januari 1–december 31). Dagwaarden van weerstations [De Bilt]. Ministerie van Infrastructuur en Waterstaat. https://daggegevens.knmi.nl/

KNMI. (1911–2021a). Zware stormen in Nederland sinds 1910 [Dataset]. Ministerie van Infrastructuur en Waterstaat. https://www.knmi.nl/nederland-nu/klimatologie/lijsten/zwarestormen

In [136]:
from numpy import NaN
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split #We need this to split the data
from sklearn.neighbors import KNeighborsClassifier #the object class we need
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Datasets

Lees de daggegevens van knmi in

In [137]:
daggegevensDf = pd.read_csv("daggegevens.csv", skiprows=46, index_col=0) # Sla de eerste 46 regels over
print(daggegevensDf)

       YYYYMMDD  DDVEC  FHVEC     FG    FHX   FHXH    FHN   FHNH    FXX  \
# STN                                                                     
260    19110101    254     46     57     72      9     31     24          
260    19110102    281     21     31     46      1     10     21          
260    19110103     51     51     57     72     17     41      4          
260    19110104     50     51     51     82     15     36     21          
260    19110105     51     31     31     51     12     10     23          
...         ...    ...    ...    ...    ...    ...    ...    ...    ...   
260    20211227    141     33     33     40     11     30      1     80   
260    20211228    216     29     45     70     13     30      1    170   
260    20211229    231     30     37     50     18     20      9    100   
260    20211230    236     64     64     80     12     50      2    150   
260    20211231    227     55     55     70      7     40     17    140   

        FXXH  ...   VVNH

C:\Users\danie\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,13,15,16,17,18,19,20,21,22,23,24,27,29,30,31,32,33,34,36,37,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Scrape de zware stormen

In [138]:

url = "https://www.knmi.nl/nederland-nu/klimatologie/lijsten/zwarestormen" 
response = requests.get(url) 
html = BeautifulSoup(response.text, "html.parser") 
table = html.find_all('table')

stormenDf = pd.read_html(str(table)) # Lees de tabel in
stormenDf = pd.DataFrame(stormenDf[0])
stormenList =  []

for index, row in stormenDf.iterrows():
    stormenList.append([row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9]]) # Voeg de tabel toe aan de stormenList

stormenDf = pd.DataFrame(stormenList) # Zet de stormenList om naar een DataFrame
stormenDf.to_csv("stormen.csv", index=False)

In [139]:
# Maanden omzetten naar cijfers
for col in stormenDf:
    stormenDf[1] = stormenDf[1].str.replace(' jan', '01')
    stormenDf[1] = stormenDf[1].str.replace(' feb', '02')
    stormenDf[1] = stormenDf[1].str.replace(' mrt', '03')
    stormenDf[1] = stormenDf[1].str.replace(' apr', '04')
    stormenDf[1] = stormenDf[1].str.replace(' mei', '05')
    stormenDf[1] = stormenDf[1].str.replace(' jun', '06')
    stormenDf[1] = stormenDf[1].str.replace(' jul', '07')
    stormenDf[1] = stormenDf[1].str.replace(' aug', '08')
    stormenDf[1] = stormenDf[1].str.replace(' sep', '09')
    stormenDf[1] = stormenDf[1].str.replace(' okt', '10')
    stormenDf[1] = stormenDf[1].str.replace(' nov', '11')
    stormenDf[1] = stormenDf[1].str.replace(' dec', '12')

# Nul toevoegen aan ééncijferige data (bijvoorbeeld 1 oktober, 110 --> 0101)
for index, row in stormenDf.iterrows():
    if len(row[1]) < 4:
        stormenDf.iat[index, 1] = '0' + stormenDf.iloc[index][1]

print(stormenDf)
stormenDf.to_csv('stormen2.csv', index=False)

       0           1   2   3    4     5      6                 7      8
0   1911  3009/  110  11  30  108  38.0  137.0  Hoek van Holland    NaN
1   1912        2708  10  27   97  41.0  148.0  Hoek van Holland    NaN
2   1913     26/2712  10  28  101   NaN    NaN        Vlissingen    NaN
3   1914        1208  10  26   94   NaN    NaN        Den Helder    NaN
4   1914        1111  10  26   94   NaN    NaN        Vlissingen    NaN
..   ...         ...  ..  ..  ...   ...    ...               ...    ...
61  2016        2011  10  26   94  37.0  133.0          IJmuiden    NaN
62  2017        1309  10  26   94  35.0  126.0          Vlieland    NaN
63  2018        0301  10  26   94  34.0  122.0          Vlieland    NaN
64  2018        1801  11  30  108  40.0  144.0  Hoek van Holland    NaN
65  2020        0902  10  25   90  36.0  130.0          Vlieland  Ciara

[66 rows x 9 columns]


Sommige stormen duren twee dagen. Ik bewerk het csv bestand handmatig: stond er bijvoorbeeld eerst als datum "3009/   110", dan worden dat nu twee rijen met datum 30-09 en 01-10.

In [140]:
stormenDf3 = pd.read_csv ('stormen3.csv', dtype=str)

stormenDf3 = stormenDf3.rename(columns={'0': 'YYYYMMDD',
                                        '1': 'Datum',
                                        '2': 'Hoogste windsnelheid Bft',
                                        '3': 'Hoogste uur m/s',
                                        '4': 'Hoogste uur km/h',
                                        '5': 'Zwaarste windstoot m/s',
                                        '6': 'Zwaarste windstoot km/h',
                                        '7': 'Plaats',
                                        '8': 'Naam',})

De kolom jaartal en datum worden samengevoegd zodat hij overeenkomt met dagwaarden dataframe: YYYYMMDD

In [141]:
stormenDf3['Date'] = stormenDf3['Datum'].str[:2]
stormenDf3['Month'] = stormenDf3['Datum'].str[2:]
stormenDf3['YYYYMMDD'] = stormenDf3['YYYYMMDD'] + stormenDf3['Month'] + stormenDf3['Date']

del stormenDf3['Datum']
del stormenDf3['Month']
del stormenDf3['Date']

print(stormenDf3)

    YYYYMMDD Hoogste windsnelheid Bft Hoogste uur m/s Hoogste uur km/h  \
0   19110930                       11              30              108   
1   19111001                       11              30              108   
2   19120827                       10              27               97   
3   19131226                       10              28              101   
4   19131227                       10              28              101   
..       ...                      ...             ...              ...   
73  20161120                       10              26               94   
74  20170913                       10              26               94   
75  20180103                       10              26               94   
76  20180118                       11              30              108   
77  20200209                       10              25               90   

   Zwaarste windstoot m/s Zwaarste windstoot km/h            Plaats   Naam  
0                    38.0         

Ik maak een nieuwe kolom die bijhoudt of het heeft gestormd die dag.

In [142]:
stormTrueOrFalse = []
for i in range(len(stormenDf3)):
    stormTrueOrFalse.append(1) # Voeg telkens een 1 toe bij de stormen

stormenDf3.insert(1, 'Storm', stormTrueOrFalse)
stormenDf3['Storm'] = stormenDf3['Storm'].astype(int) # Sla de value op als boolean

Datasets samenvoegen

In [143]:
daggegevensDf['YYYYMMDD'] = daggegevensDf['YYYYMMDD'].astype(str)

df = stormenDf3.merge(daggegevensDf, how='right', left_on='YYYYMMDD', right_on='YYYYMMDD')
df['Storm'] = df['Storm'].fillna(0).astype(int) # Verander NaN in False

df.columns = df.columns.str.replace(' ', '') # Verwijder overige spaties in kolomnamen
df = df.replace(' ', '', regex=True) # Verwijder alle spaties

df.to_csv('daggegevens_en_stormen.csv', index=False)
df.info()
df.head(30) # Laat de dataset zien

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40543 entries, 0 to 40542
Data columns (total 48 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   YYYYMMDD                40543 non-null  object
 1   Storm                   40543 non-null  int32 
 2   HoogstewindsnelheidBft  78 non-null     object
 3   Hoogsteuurm/s           78 non-null     object
 4   Hoogsteuurkm/h          78 non-null     object
 5   Zwaarstewindstootm/s    63 non-null     object
 6   Zwaarstewindstootkm/h   63 non-null     object
 7   Plaats                  78 non-null     object
 8   Naam                    1 non-null      object
 9   DDVEC                   40543 non-null  object
 10  FHVEC                   40543 non-null  object
 11  FG                      40543 non-null  object
 12  FHX                     40543 non-null  object
 13  FHXH                    40543 non-null  object
 14  FHN                     40543 non-null  object
 15  FH

,YYYYMMDD,Storm,HoogstewindsnelheidBft,Hoogsteuurm/s,Hoogsteuurkm/h,Zwaarstewindstootm/s,Zwaarstewindstootkm/h,Plaats,Naam,DDVEC,...,VVNH,VVX,VVXH,NG,UG,UX,UXH,UN,UNH,EV24
0,19110101,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,254,...,,,,,87,,,,,
1,19110102,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,281,...,,,,,95,,,,,
2,19110103,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51,...,,,,,89,,,,,
3,19110104,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,...,,,,,86,,,,,
4,19110105,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51,...,,,,,90,,,,,
5,19110106,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,157,...,,,,,90,,,,,
6,19110107,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,190,...,,,,,95,,,,,
7,19110108,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185,...,,,,,98,,,,,
8,19110109,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216,...,,,,,99,,,,,
9,19110110,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,278,...,,,,,88,,,,,


# Data cleaning

*Het KNMI gaat uit van storm of windkracht 9 als een uurgemiddelde windsnelheid tussen 75 en 88 kilometer per uur (20,8 - 24,4 meter per seconde) gemeten wordt. Volgens de schaal van Beaufort is er sprake van storm als er een 10 minuut gemiddelde windsnelheid tussen 75 en 88 km/u gemeten wordt.*

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- KNMI. (z.d.). Storm. Ministerie van Infrastructuur en Waterstaat. Geraadpleegd op 11 januari 2022, van https://www.knmi.nl/kennis-en-datacentrum/uitleg/storm 

---

* *YYYYMMDD*: Jaartal/Maand/Dag
* *Storm*: Heeft het gestormd? True/False, de dependent waarde
* *Hoogste windsnelheid Bft*: bevat vrijwel alleen maar NaN en is niet waardevol voor het onderzoek
* *Hoogste uur m/s*: bevat vrijwel alleen maar NaN en is niet waardevol voor het onderzoek
* *Hoogste uur km/h*: bevat vrijwel alleen maar NaN en is niet waardevol voor het onderzoek
* *Zwaarste windstoot m/s*: bevat vrijwel alleen maar NaN en is niet waardevol voor het onderzoek
* *Zwaarste windstoot km/h*: bevat vrijwel alleen maar NaN en is niet waardevol voor het onderzoek
* *Plaats*: bevat vrijwel alleen maar NaN en is niet waardevol voor het onderzoek
* *Naam*: bevat vrijwel alleen maar NaN en is niet waardevol voor het onderzoek
* *(DDVEC) Vectorgemiddelde windrichting in graden (360=noord, 90=oost, 180=zuid, 270=west, 0=windstil/variabel*: irrelevant
* *(FHVEC) Vectorgemiddelde windsnelheid (in 0.1 m/s)*: relevant
* *(FG) Etmaalgemiddelde windsnelheid (in 0.1 m/s)*: relevant
* *(FHX) Hoogste uurgemiddelde windsnelheid (in 0.1 m/s)*: relevant
* *(FHXH) Uurvak waarin FHX is gemeten*: irrelevant
* *(FHN) Laagste uurgemiddelde windsnelheid (in 0.1 m/s)*: irrelevant
* *(FHNH) Uurvak waarin FHN is gemeten*: irrelevant
* *(FXX) Hoogste windstoot (in 0.1 m/s)*: relevant, maar heeft veel NaN dus niet waardevol voor het onderzoek
* *(FXXH) Uurvak waarin FXX is gemeten*: irrelevant
* *(TG) Etmaalgemiddelde temperatuur (in 0.1 graden Celsius)*: irrelevant
* *(TN) Minimum temperatuur (in 0.1 graden Celsius)*: irrelevant
* *(TNH) Uurvak waarin TN is gemeten*: irrelevant
* *(TX) Maximum temperatuur (in 0.1 graden Celsius)*: irrelevant
* *(TXH) Uurvak waarin TX is gemeten*: irrelevant
* *(T10N) Minimum temperatuur op 10 cm hoogte (in 0.1 graden Celsius)*: irrelevant
* *(T10NH) 6-uurs tijdvak waarin T10N is gemeten 6=0-6 UT, 12=6-12 UT, 18=12-18 UT, 24=18-24 UT*: irrelevant
* *(SQ) Zonneschijnduur (in 0.1 uur) berekend uit de globale straling (-1 voor <0.05 uur)*: irrelevant
* *(SP) Percentage van de langst mogelijke zonneschijnduur*: irrelevant
* *(Q) Globale straling (in J/cm2)*: irrelevant
* *(DR) Duur van de neerslag (in 0.1 uur)*: irrelevant
* *(RH) Etmaalsom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm)*: irrelevant
* *(RHX) Hoogste uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm)*: irrelevant
* *(RHXH) Uurvak waarin RHX is gemeten*: irrelevant
* *(PG) Etmaalgemiddelde luchtdruk herleid tot zeeniveau (in 0.1 hPa) berekend uit 24 uurwaarden*: irrelevant
* *(PX) Hoogste uurwaarde van de luchtdruk herleid tot zeeniveau (in 0.1 hPa)*: irrelevant
* *(PXH) Uurvak waarin PX is gemeten / Hourly division in which PX was measured*: irrelevant
* *(PN) Laagste uurwaarde van de luchtdruk herleid tot zeeniveau (in 0.1 hPa)*: irrelevant
* *(PNH) Uurvak waarin PN is gemeten / Hourly division in which PN was measured*:irrelevant
* *(VVN) Minimum opgetreden zicht; 0: <100 m, 1:100-200 m, 2:200-300 m,..., 49:4900-5000 m, 50:5-6 km, 56:6-7 km, 57:7-8 km,..., 79:29-30 km, 80:30-35 km, 81:35-40 km,..., 89: >70 km*: irrelevant
* *(VVNH) Uurvak waarin VVN is gemeten / Hourly division in which VVN was measured*: irrelevant
* *(VVX) Maximum opgetreden zicht; 0: <100 m, 1:100-200 m, 2:200-300 m,..., 49:4900-5000 m, 50:5-6 km, 56:6-7 km, 57:7-8 km,..., 79:29-30 km, 80:30-35 km, 81:35-40 km,..., 89: >70 km)*: irrelevant
* *(VVXH) Uurvak waarin VVX is gemeten*: irrelevant
* *(NG) Etmaalgemiddelde bewolking (bedekkingsgraad van de bovenlucht in achtsten, 9=bovenlucht onzichtbaar)*: irrelevant
* *(UG) Etmaalgemiddelde relatieve vochtigheid (in procenten)*: irrelevant
* *(UX) Maximale relatieve vochtigheid (in procenten)*: irrelevant
* *(UXH) Uurvak waarin UX is gemeten*: irrelevant
* *(UN) Minimale relatieve vochtigheid (in procenten)*: irrelevant
* *(UNH) Uurvak waarin UN is gemeten*: irrelevant
* *(EV24) Referentiegewasverdamping (Makkink) (in 0.1 mm)*: irrelevant

Onderstaande waarden zijn relevant voor ons onderzoek:
* We gaan werken met: *YYYYMMDD*, *Storm*, *FHVEC* (Vectorgemiddelde windsnelheid (in 0.1 m/s)), *FG* (Etmaalgemiddelde windsnelheid (in 0.1 m/s)), *FHX* (Hoogste uurgemiddelde windsnelheid (in 0.1 m/s))

In [144]:
df = df[['YYYYMMDD','Storm', 'FHVEC', 'FG', 'FHX']]
#df = df[['Storm', 'FHVEC', 'FG', 'FHX']]


#df['YYYYMMDD'] = df['YYYYMMDD'].astype('str')
#df['YYYYMMDD'] = df['YYYYMMDD'].str.replace(',', '')
#df['YYYYMMDD'] = pd.to_numeric(df['YYYYMMDD'], errors='coerce')

df['YYYYMMDD'] = df['YYYYMMDD'].astype('string')
df['FHVEC'] = pd.to_numeric(df['FHVEC'])
df['FG'] = pd.to_numeric(df['FG'])
df['FHX'] = pd.to_numeric(df['FHX'])

#df['FHVEC'] = df['FHVEC'].astype(int)
#df['FHVEC'] = df.FHVEC.astype(int)
# df['FG'] = df['FG'].astype(int)
# df['FHX'] = df['FHX'].astype(int)

df.convert_dtypes().dtypes    

print(df.head(30))
df.info()
df.dtypes

    YYYYMMDD  Storm  FHVEC    FG    FHX
0   19110101      0   46.0  57.0   72.0
1   19110102      0   21.0  31.0   46.0
2   19110103      0   51.0  57.0   72.0
3   19110104      0   51.0  51.0   82.0
4   19110105      0   31.0  31.0   51.0
5   19110106      0   41.0  46.0   98.0
6   19110107      0   51.0  57.0   77.0
7   19110108      0   72.0  72.0   87.0
8   19110109      0   41.0  57.0   77.0
9   19110110      0   41.0  51.0   67.0
10  19110111      0   87.0  87.0  118.0
11  19110112      0    5.0  62.0   87.0
12  19110113      0   21.0  36.0   51.0
13  19110114      0   15.0  21.0   31.0
14  19110115      0   21.0  21.0   31.0
15  19110116      0   10.0  26.0   36.0
16  19110117      0   31.0  31.0   46.0
17  19110118      0   41.0  41.0   51.0
18  19110119      0   31.0  31.0   41.0
19  19110120      0    5.0  21.0   36.0
20  19110121      0   26.0  31.0   41.0
21  19110122      0   26.0  26.0   36.0
22  19110123      0   26.0  31.0   41.0
23  19110124      0   41.0  41.0   67.0


YYYYMMDD     string
Storm         int32
FHVEC       float64
FG          float64
FHX         float64
dtype: object

In [145]:
df = df.dropna() # Verwijder rijen met lege cellen
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40513 entries, 0 to 40542
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   YYYYMMDD  40513 non-null  string 
 1   Storm     40513 non-null  int32  
 2   FHVEC     40513 non-null  float64
 3   FG        40513 non-null  float64
 4   FHX       40513 non-null  float64
dtypes: float64(3), int32(1), string(1)
memory usage: 1.7 MB


Het aantal stormen dat er geweest zijn is 78.

In [146]:
df['Storm'].value_counts()

0    40435
1       78
Name: Storm, dtype: int64

Let's add dummy variables for the variable *Sex*. Remember we can only add one of the variables *male* and *female*. They are perfectly correlated in this dataset so the model wouldn't be able to distinguish between them.

In [147]:
df = pd.get_dummies(df,columns=['FHVEC'],drop_first= True)
#df = df.join(pd.get_dummies(df['FHVEC']))
df.head()
#df.to_csv('test.csv')

df.dtypes
df.head(30) # Laat de dataset zien

,YYYYMMDD,Storm,FG,FHX,FHVEC_1.0,FHVEC_2.0,FHVEC_3.0,FHVEC_4.0,FHVEC_5.0,FHVEC_6.0,...,FHVEC_106.0,FHVEC_108.0,FHVEC_113.0,FHVEC_118.0,FHVEC_123.0,FHVEC_129.0,FHVEC_134.0,FHVEC_139.0,FHVEC_144.0,FHVEC_149.0
0,19110101,0,57.0,72.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,19110102,0,31.0,46.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,19110103,0,57.0,72.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,19110104,0,51.0,82.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,19110105,0,31.0,51.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,19110106,0,46.0,98.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,19110107,0,57.0,77.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,19110108,0,72.0,87.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,19110109,0,57.0,77.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,19110110,0,51.0,67.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Het model bouwen

In [148]:
X = df.loc[:, ~df.columns.isin(['Storm'])] # Create the X matrix
y = df['Storm'] # Create the y-variable

X = normalize(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
pd.DataFrame(X_train).head()

,0,1,2,3,4,5,6,7,8,9,...,101,102,103,104,105,106,107,108,109,110
0,1.0,0.000004,0.000006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.000002,0.000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.000003,0.000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.000001,0.000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.000002,0.000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [149]:
knn = KNeighborsClassifier() #create a KNN-classifier with 5 neighbors (default)
knn = knn.fit(X_train, y_train) #this fits the k-nearest neigbor model with the train data

# Model evaluatie
Ik start met het uitrekenen van de accuraatheid.

In [150]:
knn.score(X_test, y_test) # calculate the fit on the *test* data

0.9976139542537437

De accuraatheid is erg hoog: 99.8%. 

We kunnen dit vergelijken met de best baseline guess: altijd "geen storm". Dat betekent 40435 / (40435 + 78) * 100 = 99,8% (zie *value_counts* hier boven). Het model is evengoed als de baseline guess.

In [151]:
# Model evaluatie: precizie en recall

In [152]:
y_test_pred = knn.predict(X_test) #the predicted values
cm = confusion_matrix(y_test, y_test_pred) #creates a "confusion matrix"
cm

array([[12122,     7],
       [   22,     3]], dtype=int64)

In [153]:
conf_matrix = pd.DataFrame(cm, index=['Geen storm (actual)', 'Wel storm (actual)'], columns = ['Geen storm (voorspelling)', 'Wel storm (voorspelling)']) 
conf_matrix

,Geen storm (voorspelling),Wel storm (voorspelling)
Geen storm (actual),12122,7
Wel storm (actual),22,3


Dit betekent dat:

Wanneer de voorspelling is dat er 12122 dagen er geen storm is en

?????????????

## Accuraatheid
Ik reken de accuraatheiduit aan de hand van de confusion matrix.

In [154]:
(12122+3)/(12122+22+7+3)

0.9976139542537437

De accuraatheid is inderdaad 99.8%

## Precision (wél storm)

In [155]:
3/(7+3)

0.3

De precision is 0.3%, dus extreem laag.

## Recall (survived)

In [156]:
3/(22+3)

0.12

Dus 0.12%. Alweer extreem laag. Het model is beter in het schatten van geen storm dan in het schatten van wel storm. Dit is logisch omdat een storm slechts 78 / (78 + 40435) * 100 = 0,2% van de tijd voorkomt. De kans is gewoon veel groter dat er geen storm is.

# Parameter setting

In [157]:
for i in range(1,11):
    knn_new = KNeighborsClassifier(n_neighbors = i) #make a new kNN model with i (1-10) neighbors
    knn_new = knn_new.fit(X_train, y_train) #fit new model on train data
    y_test_pred_new = knn_new.predict(X_test) #predict using new model, with test data
    print(f"With {i} neighbors the result is:")
    print(classification_report(y_test, y_test_pred_new)) #use a built-in function to print out accuracy, precision and recall


With 1 neighbors the result is:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12129
           1       0.33      0.12      0.18        25

    accuracy                           1.00     12154
   macro avg       0.67      0.56      0.59     12154
weighted avg       1.00      1.00      1.00     12154

With 2 neighbors the result is:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12129
           1       0.33      0.04      0.07        25

    accuracy                           1.00     12154
   macro avg       0.67      0.52      0.54     12154
weighted avg       1.00      1.00      1.00     12154

With 3 neighbors the result is:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12129
           1       0.40      0.16      0.23        25

    accuracy                           1.00     12154
   macro avg       0.70      0.5